# Module 10 Application

## Challenge: Crypto Investments

In this Challenge, you’ll combine your financial Python programming skills with the new unsupervised learning skills that you acquired in this module.

The CSV file provided for this challenge contains price change data of cryptocurrencies in different periods.

The steps for this challenge are broken out into the following sections:

* Import the Data (provided in the starter code)
* Prepare the Data (provided in the starter code)
* Cluster Cryptocurrencies with K-means
* Find the Best Value for k
* Optimize Clusters with Principal Component Analysis
* Visualize the Results

### Import the Data

This section imports the data into a new DataFrame. It follows these steps:

1. Read  the “crypto_market_data.csv” file from the Resources folder into a DataFrame, and use `index_col="coin_id"` to set the cryptocurrency name as the index. Review the DataFrame.

2. Generate the summary statistics, and and use HvPlot to visualize your data to observe what your DataFrame contains.


> **Rewind:** The [Pandas`describe()`function](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.describe.html) generates summary statistics for a DataFrame. 

In [1]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from path import Path
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [2]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    Path("Resources/crypto_market_data.csv"),
    index_col="coin_id")

# Display sample data
df_market_data.head(10)

price_change_percentage_24h  price_change_percentage_7d  \
coin_id                                                                    
bitcoin                              1.08388                     7.60278   
ethereum                             0.22392                    10.38134   
tether                              -0.21173                     0.04935   
ripple                              -0.37819                    -0.60926   
bitcoin-cash                         2.90585                    17.09717   
binancecoin                          2.10423                    12.85511   
chainlink                           -0.23935                    20.69459   
cardano                              0.00322                    13.99302   
litecoin                            -0.06341                     6.60221   
bitcoin-cash-sv                      0.92530                     3.29641   

                 price_change_percentage_14d  price_change_percentage_30d  \
coin_id                                                                     
bitcoin                              6.57509                      7.67258   
ethereum                             4.80849                      0.13169   
tether                               0.00640                     -0.04237   
ripple                               2.24984                      0.23455   
bitcoin-cash                        14.75334                     15.74903   
binancecoin                          6.80688                      0.05865   
chainlink                            9.30098                    -11.21747   
cardano                              5.55476                     10.10553   
litecoin                             7.28931                      1.21662   
bitcoin-cash-sv                     -1.86656                      2.88926   

                 price_change_percentage_60d  price_change_percentage_200d  \
coin_id                                                                      
bitcoin                             -3.25185                      83.51840   
ethereum                           -12.88890                     186.77418   
tether                               0.28037                      -0.00542   
ripple                             -17.55245                      39.53888   
bitcoin-cash                       -13.71793                      21.66042   
binancecoin                         36.33486                     155.61937   
chainlink                          -43.69522                     403.22917   
cardano                            -22.84776                     264.51418   
litecoin                           -17.23960                      27.49919   
bitcoin-cash-sv                    -24.87434                       7.42562   

                 price_change_percentage_1y  
coin_id                                      
bitcoin                            37.51761  
ethereum                          101.96023  
tether                              0.01954  
ripple                            -16.60193  
bitcoin-cash                       14.49384  
binancecoin                        69.69195  
chainlink                         325.13186  
cardano                           156.09756  
litecoin                          -12.66408  
bitcoin-cash-sv                    93.73082

In [3]:
# Generate summary statistics
df_market_data.describe()

price_change_percentage_24h  price_change_percentage_7d  \
count                    41.000000                   41.000000   
mean                     -0.269686                    4.497147   
std                       2.694793                    6.375218   
min                     -13.527860                   -6.094560   
25%                      -0.608970                    0.047260   
50%                      -0.063410                    3.296410   
75%                       0.612090                    7.602780   
max                       4.840330                   20.694590   

       price_change_percentage_14d  price_change_percentage_30d  \
count                    41.000000                    41.000000   
mean                      0.185787                     1.545693   
std                       8.376939                    26.344218   
min                     -18.158900                   -34.705480   
25%                      -5.026620                   -10.438470   
50%                       0.109740                    -0.042370   
75%                       5.510740                     4.578130   
max                      24.239190                   140.795700   

       price_change_percentage_60d  price_change_percentage_200d  \
count                    41.000000                     41.000000   
mean                     -0.094119                    236.537432   
std                      47.365803                    435.225304   
min                     -44.822480                     -0.392100   
25%                     -25.907990                     21.660420   
50%                      -7.544550                     83.905200   
75%                       0.657260                    216.177610   
max                     223.064370                   2227.927820   

       price_change_percentage_1y  
count                   41.000000  
mean                   347.667956  
std                   1247.842884  
min                    -17.567530  
25%                      0.406170  
50%                     69.691950  
75%                    168.372510  
max                   7852.089700

In [4]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

:NdOverlay   [Variable]
   :Curve   [coin_id]   (value)

### Prepare the Data

This section prepares the data before running the K-Means algorithm. It follows these steps:

1. Use the `StandardScaler` module from scikit-learn to normalize the CSV file data. This will require you to utilize the `fit_transform` function.

2. Create a DataFrame that contains the scaled data. Be sure to set the `coin_id` index from the original DataFrame as the index for the new DataFrame. Review the resulting DataFrame.


In [5]:
#the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
scaled_data = StandardScaler().fit_transform(df_market_data)

In [6]:
# DataFrame with the scaled data
df_market_data_scaled = pd.DataFrame(
    scaled_data,
    columns=df_market_data.columns
)

# the crypto names from the original data
df_market_data_scaled["coin_id"] = df_market_data.index

# the coinid column as index
df_market_data_scaled = df_market_data_scaled.set_index("coin_id")

# Display sample data
df_market_data_scaled.head()

price_change_percentage_24h  price_change_percentage_7d  \
coin_id                                                                 
bitcoin                          0.508529                    0.493193   
ethereum                         0.185446                    0.934445   
tether                           0.021774                   -0.706337   
ripple                          -0.040764                   -0.810928   
bitcoin-cash                     1.193036                    2.000959   

              price_change_percentage_14d  price_change_percentage_30d  \
coin_id                                                                  
bitcoin                          0.772200                     0.235460   
ethereum                         0.558692                    -0.054341   
tether                          -0.021680                    -0.061030   
ripple                           0.249458                    -0.050388   
bitcoin-cash                     1.760610                     0.545842   

              price_change_percentage_60d  price_change_percentage_200d  \
coin_id                                                                   
bitcoin                         -0.067495                     -0.355953   
ethereum                        -0.273483                     -0.115759   
tether                           0.008005                     -0.550247   
ripple                          -0.373164                     -0.458259   
bitcoin-cash                    -0.291203                     -0.499848   

              price_change_percentage_1y  
coin_id                                   
bitcoin                        -0.251637  
ethereum                       -0.199352  
tether                         -0.282061  
ripple                         -0.295546  
bitcoin-cash                   -0.270317

---

# Cluster Cryptocurrencies with K-means

In this section, you will use the K-Means algorithm with a given value for `k` to cluster the cryptocurrencies according to the price changes of cryptocurrencies provided.

1. Initialize the K-Means model with four clusters (`n_clusters=4`). 

2. Fit the K-Means model using the scaled data.

3. Predict the clusters to group the cryptocurrencies using the scaled data. View the resulting array of cluster values.

4. Add a new column to the DataFrame with the scaled data with the predicted clusters.

5. Create a scatter plot using hvPlot by setting `x="price_change_percentage_14d"` and `y="price_change_percentage_1y"`. Color the graph points with the labels found using K-Means and add the crypto name in the `hover_cols` parameter to identify the cryptocurrency represented by each data point.

In [7]:
# Initialize the K-Means model with four clusters
model = KMeans(n_clusters=4, random_state=1)

# Print the model
model

KMeans(n_clusters=4, random_state=1)

In [8]:
# Fit the K-Means model using the scaled data
model.fit(df_market_data_scaled)

KMeans(n_clusters=4, random_state=1)

In [9]:
# Predict the clusters to group the cryptocurrencies using the scaled data
crypto_clusters_k4 = model.predict(df_market_data_scaled)

# View the resulting array of cluster values.
print(crypto_clusters_k4)

[3 3 1 1 3 3 3 3 3 1 1 1 1 3 1 3 1 1 3 1 1 3 1 1 1 1 1 1 3 1 1 1 0 3 1 1 2
 1 1 1 1]


In [10]:
# Note: The code for this step is provided for you.

# Add a new column to the DataFrame with the predicted clusters with k=4
df_market_data_scaled["crypto_cluster_k4"] = crypto_clusters_k4

# Display sample data
df_market_data_scaled.head()

price_change_percentage_24h  price_change_percentage_7d  \
coin_id                                                                 
bitcoin                          0.508529                    0.493193   
ethereum                         0.185446                    0.934445   
tether                           0.021774                   -0.706337   
ripple                          -0.040764                   -0.810928   
bitcoin-cash                     1.193036                    2.000959   

              price_change_percentage_14d  price_change_percentage_30d  \
coin_id                                                                  
bitcoin                          0.772200                     0.235460   
ethereum                         0.558692                    -0.054341   
tether                          -0.021680                    -0.061030   
ripple                           0.249458                    -0.050388   
bitcoin-cash                     1.760610                     0.545842   

              price_change_percentage_60d  price_change_percentage_200d  \
coin_id                                                                   
bitcoin                         -0.067495                     -0.355953   
ethereum                        -0.273483                     -0.115759   
tether                           0.008005                     -0.550247   
ripple                          -0.373164                     -0.458259   
bitcoin-cash                    -0.291203                     -0.499848   

              price_change_percentage_1y  crypto_cluster_k4  
coin_id                                                      
bitcoin                        -0.251637                  3  
ethereum                       -0.199352                  3  
tether                         -0.282061                  1  
ripple                         -0.295546                  1  
bitcoin-cash                   -0.270317                  3

In [11]:
# Create a scatter plot using hvPlot by setting 
# `x="price_change_percentage_14d"` and `y="price_change_percentage_1y"`.
# Group the results by the clusters using `by="crypto_cluster_k4".
# Set the hover to the coin id using `hover_cols=["coin_id"]`.

df_market_data_scaled.hvplot.scatter(
    x="price_change_percentage_14d",
    y="price_change_percentage_1y",
    by="crypto_cluster_k4",
    hover_cols=["coin_id"],
    marker=["hex", "square", "cross", "inverted_triangle", "triangle"],
)

:NdOverlay   [crypto_cluster_k4]
   :Scatter   [price_change_percentage_14d]   (price_change_percentage_1y,coin_id)

---

# Find the Best Value for k

In this section, you will use the elbow method to find the best value for k.

1. Code the elbow method algorithm to find the best value for k. Use a range from 1 to 11. 

2. Plot a line chart with all the inertia values computed with the different values of k to visually identify the optimal value for k.

3. Answer the following question: What is the best value for k?

In [12]:
# Create a list with the number of k-values to try
# Use a range from 1 to 11
k = list(range(1, 11))

In [13]:
# Create an empy list to store the inertia values
inertia = []

In [14]:
# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_scaled`
# 3. Append the model.inertia_ to the inirtia list

for i in k:
    k_model = KMeans(n_clusters=i, random_state=1)
    k_model.fit(df_market_data_scaled)
    inertia.append(k_model.inertia_)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:881: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


In [15]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data = {
    "k": k,
    "inertia": inertia
}

# Create a DataFrame with the data to plot the Elbow curve
df_elbow = pd.DataFrame(elbow_data)

In [16]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
df_elbow.hvplot.line(
    x="k", 
    y="inertia", 
    title="Elbow Curve", 
    xticks=k
)

:Curve   [k]   (inertia)

#### 3. Answer the following question: What is the best value for k?
**Question:** What is the best value for `k`?

**Answer: 4

---

# Optimize Clusters with Principal Component Analysis

In this section, you will perform a principal component analysis (PCA) and reduce the features to three principal components.

1. Create a PCA model instance and set `n_components=3`.

2. Use the PCA model to reduce to three principal components. View the first five rows of the DataFrame. 

3. Retrieve the explained variance to determine how much information can be attributed to each principal component.

4. Answer the following question: What is the total explained variance of the three principal components?

5. Create a new DataFrame with the PCA data. Be sure to set the `coin_id` index from the original DataFrame as the index for the new DataFrame. Review the resulting DataFrame.

6. Initiate a new  K-Means algorithm using the PCA DataFrame to group the cryptocurrencies. Set the `n_components` parameter equal to the best value for `k` found before. View the resulting array. 

7. For further analysis, add the following columns to the DataFrame with the PCA data. Review the resulting DataFrame once the additional columns have been added. Make sure to do the following:

    - From the original DataFrame, add the `price_change_percentage_1y` and `price_change_percentage_14d` columns.

    - Add a column with the predicted cluster values identified using a k value of 4. (The predicted cluster values were calculated in the “Cluster Cryptocurrencies with K-means” section.)

    - Add a column with the predicted cluster values identified using the optimal value for k. 


In [17]:
# Create a PCA model instance and set `n_components=3`.
pca = PCA(n_components=3)


In [18]:
# Use the PCA model with `fit_transform` to reduce to 
# three principal components.
df_market_data_scaled1 = pca.fit_transform(df_market_data_scaled)


# View the first five rows of the DataFrame. 
df_market_data_scaled1[:5]

array([[-1.58513942,  0.13439444,  0.545621  ],
       [-1.32258066,  0.05449393,  1.11570854],
       [ 0.26633536, -0.53640937, -0.79694588],
       [ 0.25722513, -0.59201286, -0.6465287 ],
       [-2.95224985,  0.42237155,  1.26789353]])

In [19]:
# Retrieve the explained variance to determine how much information 
# can be attributed to each principal component.
pca.explained_variance_ratio_


array([0.37005408, 0.32322221, 0.19115222])

In [20]:
pca.explained_variance_ratio_.sum()

0.8844285111826466

Answer the following question: What is the total explained variance of the three principal components?

**Question** What is the total explained variance of the three principal components?

**Answer** 0.88811018

In [26]:
# Create a new DataFrame with the PCA data.
# Note: The code for this step is provided for you

# Creating a DataFrame with the PCA data
df_market_data_pca = pd.DataFrame(
    df_market_data_scaled1, 
    columns=["PC1", "PC2", "PC3"]
)

# Copy the crypto names from the original data
df_market_data_pca["coin_id"] = df_market_data.index

# Set the coinid column as index
df_market_data_pca = df_market_data_pca.set_index("coin_id")

# Display sample data
df_market_data_pca.head()

PC1       PC2       PC3
coin_id                                   
bitcoin      -1.585139  0.134394  0.545621
ethereum     -1.322581  0.054494  1.115709
tether        0.266335 -0.536409 -0.796946
ripple        0.257225 -0.592013 -0.646529
bitcoin-cash -2.952250  0.422372  1.267894

In [27]:
# Initiate a new  K-Means algorithm using the PCA DataFrame to group 
# the cryptocurrencies. Set the `n_components` parameter equal to 
# the best value for `k` found before. View the resulting array. 
model = KMeans(n_clusters=4, random_state=1)

# Initialize the K-Means model
# YOUR CODE HERE!

# Fit the model
model.fit(df_market_data_pca)

# Predict clusters
k_4 = model.predict(df_market_data_pca)

# View the resulting array
df_market_data_pca['customer_segments_4'] = k_4
df_market_data_pca

PC1       PC2       PC3  customer_segments_4
coin_id                                                                
bitcoin              -1.585139  0.134394  0.545621                    1
ethereum             -1.322581  0.054494  1.115709                    1
tether                0.266335 -0.536409 -0.796946                    0
ripple                0.257225 -0.592013 -0.646529                    0
bitcoin-cash         -2.952250  0.422372  1.267894                    1
binancecoin          -1.975484  0.528219  0.670145                    1
chainlink            -1.671906  0.178423  2.718598                    1
cardano              -1.539740  0.324517  1.497226                    1
litecoin             -1.347533 -0.132837  0.791063                    1
bitcoin-cash-sv       0.040698 -0.802083 -0.465562                    0
crypto-com-chain      1.344822 -1.108679 -1.076912                    0
usd-coin              0.272121 -0.545849 -0.814012                    0
eos                   0.320673 -0.918019 -0.637918                    0
monero               -2.838740  1.895142  0.848863                    1
tron                  0.143758 -0.442297 -0.274682                    0
tezos                -0.862155 -0.757488  1.490647                    1
okb                   1.378653 -0.776673 -0.754383                    0
stellar               0.532658 -0.876910 -0.120776                    0
cosmos               -1.539433  0.352390  1.713933                    1
cdai                  0.208186 -0.588555 -0.830587                    0
neo                   0.887872 -0.947174 -0.588142                    0
wrapped-bitcoin      -1.572600  0.121709  0.530894                    1
leo-token             0.697532 -0.826190 -0.993080                    0
huobi-token           0.263251 -0.527831 -0.530039                    0
nem                   0.408806  0.707042 -1.330139                    0
binance-usd           0.246128 -0.540957 -0.809982                    0
iota                  0.186495 -0.967371  0.003609                    0
vechain               1.528493 -1.668876 -0.750235                    0
zcash                -1.347135  0.106093  1.200966                    1
theta-token           2.056222  2.034190 -1.525531                    0
dash                  0.222043 -0.817068  0.122347                    0
ethereum-classic      0.228151 -0.739212 -0.299732                    0
ethlend               7.458259  4.338598  3.751968                    2
maker                -0.991935 -0.079763  0.741439                    1
havven                2.271195 -0.622938  0.875803                    0
omisego               0.234856  0.223448 -1.390103                    0
celsius-degree-token -1.949062  7.705390 -3.110327                    3
ontology              1.510753 -1.801827 -0.193429                    0
ftx-token            -0.092618 -0.521653 -0.153336                    0
true-usd              0.229197 -0.538204 -0.805785                    0
digibyte              0.393926 -0.449545 -0.988556                    0

In [28]:
# Initiate a new  K-Means algorithm using the PCA DataFrame to group 
# the cryptocurrencies. Set the `n_components` parameter equal to 
# the best value for `k` found before. View the resulting array. 
model = KMeans(n_clusters=5, random_state=1)

# Initialize the K-Means model
# YOUR CODE HERE!

# Fit the model
model.fit(df_market_data_pca)

# Predict clusters
k_5 = model.predict(df_market_data_pca)

# View the resulting array
df_market_data_pca['customer_segments_5'] = k_5
df_market_data_pca

PC1       PC2       PC3  customer_segments_4  \
coin_id                                                                   
bitcoin              -1.585139  0.134394  0.545621                    1   
ethereum             -1.322581  0.054494  1.115709                    1   
tether                0.266335 -0.536409 -0.796946                    0   
ripple                0.257225 -0.592013 -0.646529                    0   
bitcoin-cash         -2.952250  0.422372  1.267894                    1   
binancecoin          -1.975484  0.528219  0.670145                    1   
chainlink            -1.671906  0.178423  2.718598                    1   
cardano              -1.539740  0.324517  1.497226                    1   
litecoin             -1.347533 -0.132837  0.791063                    1   
bitcoin-cash-sv       0.040698 -0.802083 -0.465562                    0   
crypto-com-chain      1.344822 -1.108679 -1.076912                    0   
usd-coin              0.272121 -0.545849 -0.814012                    0   
eos                   0.320673 -0.918019 -0.637918                    0   
monero               -2.838740  1.895142  0.848863                    1   
tron                  0.143758 -0.442297 -0.274682                    0   
tezos                -0.862155 -0.757488  1.490647                    1   
okb                   1.378653 -0.776673 -0.754383                    0   
stellar               0.532658 -0.876910 -0.120776                    0   
cosmos               -1.539433  0.352390  1.713933                    1   
cdai                  0.208186 -0.588555 -0.830587                    0   
neo                   0.887872 -0.947174 -0.588142                    0   
wrapped-bitcoin      -1.572600  0.121709  0.530894                    1   
leo-token             0.697532 -0.826190 -0.993080                    0   
huobi-token           0.263251 -0.527831 -0.530039                    0   
nem                   0.408806  0.707042 -1.330139                    0   
binance-usd           0.246128 -0.540957 -0.809982                    0   
iota                  0.186495 -0.967371  0.003609                    0   
vechain               1.528493 -1.668876 -0.750235                    0   
zcash                -1.347135  0.106093  1.200966                    1   
theta-token           2.056222  2.034190 -1.525531                    0   
dash                  0.222043 -0.817068  0.122347                    0   
ethereum-classic      0.228151 -0.739212 -0.299732                    0   
ethlend               7.458259  4.338598  3.751968                    2   
maker                -0.991935 -0.079763  0.741439                    1   
havven                2.271195 -0.622938  0.875803                    0   
omisego               0.234856  0.223448 -1.390103                    0   
celsius-degree-token -1.949062  7.705390 -3.110327                    3   
ontology              1.510753 -1.801827 -0.193429                    0   
ftx-token            -0.092618 -0.521653 -0.153336                    0   
true-usd              0.229197 -0.538204 -0.805785                    0   
digibyte              0.393926 -0.449545 -0.988556                    0   

                      customer_segments_5  
coin_id                                    
bitcoin                                 0  
ethereum                                0  
tether                                  4  
ripple                                  4  
bitcoin-cash                            0  
binancecoin                             0  
chainlink                               0  
cardano                                 0  
litecoin                                0  
bitcoin-cash-sv                         4  
crypto-com-chain                        4  
usd-coin                                4  
eos                                     4  
monero                                  0  
tron                                    4  
tezos                                   0  
okb      

In [29]:
# Note: The code for the following step has been provided for you.

# For further analysis, add the following columns to the DataFrame 
# with the PCA data. Review the resulting DataFrame once the additional 
# columns have been added. Make sure to do the following:
# - From the original DataFrame, add the `price_change_percentage_1y` and `price_change_percentage_14d` columns.
# - Add a column with the predicted cluster values identified using a k value of 4. (The predicted cluster values were calculated in the “Cluster Cryptocurrencies with K-means” section.)
# - Add a column with the predicted cluster values identified using the optimal value for k. 


# Add the price_change_percentage_1y column from the original data
df_market_data_pca["price_change_percentage_1y"] = df_market_data["price_change_percentage_1y"]

# Add the price_change_percentage_14d column from the original data
df_market_data_pca["price_change_percentage_14d"] = df_market_data["price_change_percentage_14d"]

# Add a new column to the DataFrame with the predicted clusters using the best value of k
df_market_data_pca["crypto_cluster_k5"] = k_5

# Add a new column to the DataFrame with the predicted clusters using k=4
df_market_data_pca["crypto_cluster_k4"] = k_4

# Display sample data
df_market_data_pca.head

<bound method NDFrame.head of                            PC1       PC2       PC3  customer_segments_4  \
coin_id                                                                   
bitcoin              -1.585139  0.134394  0.545621                    1   
ethereum             -1.322581  0.054494  1.115709                    1   
tether                0.266335 -0.536409 -0.796946                    0   
ripple                0.257225 -0.592013 -0.646529                    0   
bitcoin-cash         -2.952250  0.422372  1.267894                    1   
binancecoin          -1.975484  0.528219  0.670145                    1   
chainlink            -1.671906  0.178423  2.718598                    1   
cardano              -1.539740  0.324517  1.497226                    1   
litecoin             -1.347533 -0.132837  0.791063                    1   
bitcoin-cash-sv       0.040698 -0.802083 -0.465562                    0   
crypto-com-chain      1.344822 -1.108679 -1.076912                    

---

# Step 6: Plot Results

In this section, you will visually analyze the cluster analysis results after using the optimization techniques.

1. Use the PCA data to create two scatter plots using hvPlot by setting `x="price_change_percentage_14d"` and `y="price_change_percentage_1y"`. Make sure to do the following: 

    - In the first plot, color the plot points by the cluster values identified using a k value of 4.

    - In the second plot, color the plot points by the cluster values identified using the optimal value for k.

    - In both plots, add the crypto name by sing the `hover_cols` parameter to identify the cryptocurrency represented by each data point. 

2. Be sure to professionally style and format the plots so that the visualizations can be easily read. 

3. Answer the following question: What value of k creates the most accurate clusters of cryptocurrencies, grouped by profitability?


In [30]:
# Create a scatter plot for the Crypto Clusters using k=4 data.
# Use the PCA data to create a scatter plot with hvPlot by setting 
# x="price_change_percentage_14d" and y="price_change_percentage_1y".
# Group by the clusters using `by="crypto_cluster_k4".
# Set the hover colors to the coin id with `hover_cols=["coin_id"]
# Create a descriptive title for the plot using the title parameter.
scatter_plot_k4 = df_market_data_pca.hvplot.scatter(
    x="price_change_percentage_14d",
    y="price_change_percentage_1y",
    by="crypto_cluster_k4",
    hover_cols=["coin_id"],
    marker=["hex", "square", "cross", "inverted_triangle", "triangle"],
)


In [31]:
# Create a scatter plot for the Crypto Clusters using k=5 data.
# Use the PCA data to create a scatter plot with hvPlot by setting 
# x="price_change_percentage_14d" and y="price_change_percentage_1y".
# Group by the clusters using `by="crypto_cluster_k5".
# Set the hover colors to the coin id with `hover_cols=["coin_id"]
# Create a descriptive title for the plot using the title parameter.
scatter_plot_k5 = df_market_data_pca.hvplot.scatter(
    x="price_change_percentage_14d",
    y="price_change_percentage_1y",
    by="crypto_cluster_k5",
    hover_cols=["coin_id"],
    marker=["hex", "square", "cross", "inverted_triangle", "triangle"],
)

In [33]:
# Compare both scatter plots
scatter_plot_k4 + scatter_plot_k5

:Layout
   .NdOverlay.I  :NdOverlay   [crypto_cluster_k4]
      :Scatter   [price_change_percentage_14d]   (price_change_percentage_1y,coin_id)
   .NdOverlay.II :NdOverlay   [crypto_cluster_k5]
      :Scatter   [price_change_percentage_14d]   (price_change_percentage_1y,coin_id)

Answer the following question: What value of k creates the most accurate clusters of cryptocurrencies, grouped by profitability?

**Question:** What value of `k` seems to create the most accurate clusters to group cryptocurrencies according to their profitability?

**Answer:** K = 4 seems good, because in elbow curve that was an optimum number. Also, when K=5, the groups are overlapped with each other and not very segregated in comparision to K=4. Hence, K=4 is better.